In [1]:
import json
from utils.gho_filter import filter_gho_users

with open('results.json', 'r') as fp:
    res = json.load(fp)
data = []
for entry in res:
    data += entry['data']['users']

gho_users = set(filter_gho_users(data))

In [3]:
import asyncio
import os
from web3 import Web3
import concurrent
from tqdm import tqdm

provider = os.environ.get('WEB3_PROVIDER_URL')
w3 = Web3(Web3.HTTPProvider(provider))

def check_address_type(address):
    code = w3.eth.get_code(Web3.to_checksum_address(address))
    return address, len(code) > 0

def process_batch(addresses):
    return [check_address_type(addr) for addr in addresses]

def parallel_address_check(addresses, batch_size=20):
    eoa_count = 0
    contract_count = 0
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [addresses[i:i + batch_size] for i in range(0, len(addresses), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_results = future.result()
            for address, is_contract in batch_results:
                if is_contract:
                    contract_count += 1
                else:
                    eoa_count += 1
                results.append((address, 'Contract' if is_contract else 'EOA'))

    return results, eoa_count, contract_count

results, eoa_count, contract_count = parallel_address_check(list(set(gho_users)))

print(f"EOAs: {eoa_count}")
print(f"Contracts: {contract_count}")

100%|██████████████████████████████████████████████████████████████████████████████| 115/115 [00:25<00:00,  4.56it/s]

EOAs: 2176
Contracts: 121


In [4]:
121 / 2176 * 100

5.560661764705882